In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
df = pd.read_csv("/content/df_clima_Redes.csv")


In [4]:
df.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainfallTomorrow', 'DirVien_E', 'DirVien_ENE',
       'DirVien_ESE', 'DirVien_N', 'DirVien_NE', 'DirVien_NNE', 'DirVien_NNW',
       'DirVien_NW', 'DirVien_S', 'DirVien_SE', 'DirVien_SSE', 'DirVien_SSW',
       'DirVien_SW', 'DirVien_W', 'DirVien_WNW', 'DirVien_WSW', 'DirVien9am_E',
       'DirVien9am_ENE', 'DirVien9am_ESE', 'DirVien9am_N', 'DirVien9am_NE',
       'DirVien9am_NNE', 'DirVien9am_NNW', 'DirVien9am_NW', 'DirVien9am_S',
       'DirVien9am_SE', 'DirVien9am_SSE', 'DirVien9am_SSW', 'DirVien9am_SW',
       'DirVien9am_W', 'DirVien9am_WNW', 'DirVien9am_WSW', 'DirVien3am_E',
       'DirVien3am_ENE', 'DirVien3am_ESE', 'DirVien3am_N', 'DirVien3am_NE',
       'DirVien3am_NNE', 'DirVien3am_NNW', 'DirVien3am_NW', 'DirVien3am_S',
       'DirVien

In [ ]:
df.head(5)

In [7]:
scaler = MinMaxScaler()

In [9]:
columnas_nor = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm']
df[columnas_nor] = scaler.fit_transform(df[columnas_nor])

In [ ]:
df.head(5)

In [11]:
X = df[['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'DirVien_ENE',
       'DirVien_ESE', 'DirVien_N', 'DirVien_NE', 'DirVien_NNE', 'DirVien_NNW',
       'DirVien_NW', 'DirVien_S', 'DirVien_SE', 'DirVien_SSE', 'DirVien_SSW',
       'DirVien_SW', 'DirVien_W', 'DirVien_WNW', 'DirVien_WSW',
       'DirVien9am_ENE', 'DirVien9am_ESE', 'DirVien9am_N', 'DirVien9am_NE',
       'DirVien9am_NNE', 'DirVien9am_NNW', 'DirVien9am_NW', 'DirVien9am_S',
       'DirVien9am_SE', 'DirVien9am_SSE', 'DirVien9am_SSW', 'DirVien9am_SW',
       'DirVien9am_W', 'DirVien9am_WNW', 'DirVien9am_WSW',
       'DirVien3am_ENE', 'DirVien3am_ESE', 'DirVien3am_N', 'DirVien3am_NE',
       'DirVien3am_NNE', 'DirVien3am_NNW', 'DirVien3am_NW', 'DirVien3am_S',
       'DirVien3am_SE', 'DirVien3am_SSE', 'DirVien3am_SSW',
       'LlueveHoy_Yes', 'ciudades_Canberra',
       'ciudades_Melbourne', 'ciudades_MelbourneAirport', 'ciudades_Sydney',
       'ciudades_SydneyAirport'
       ]]
y = df['RainfallTomorrow']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def create_model(trial):
    model = Sequential()
    num_layers = trial.suggest_int('num_layers', 1, 5)

    for i in range(num_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128)
        model.add(Dense(num_units, activation="relu"))

    model.add(Dense(1))


    def r2_metric(y_true, y_pred):
        SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
        SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
        return 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[r2_metric])

    return model


def objective(trial):

    model = create_model(trial)
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=5, batch_size=32, verbose=0)
    _, r2 = model.evaluate(X_valid, y_valid, verbose=0)

    return r2

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print("Mejores parámetros encontrados:", best_params)

best_model = create_model(study.best_trial)